# sync

> Phase 3: Project Synchronization  
> Pull, prepare, commit, and push—the complete workflow

In [ ]:
#| default_exp sync

Sync compresses the repetitive development cycle into a single command: pull latest changes, run nbdev's prepare pipeline (export notebooks to modules, run tests, clean metadata), commit everything, and push. This workflow runs dozens of times during active development—automation prevents skipped steps and inconsistent state.

The sequence is strictly ordered and fails fast. A merge conflict aborts immediately. Test failures block the commit. Each gate ensures the next step operates on valid state.

In [ ]:
#| export
import sys
from pj.core import run_cmd, hr

## Future: Extract Git Sync Helper

**TODO: Create `git_sync()` helper for DRY**

Both `sync()` and `ship()` (in `04_ship.ipynb`) perform similar git operations:
- Add all changes
- Commit with message
- Push to remote

Extract this into:
```python
def git_sync(message, verbose=False):
    """Add, commit, and push changes"""
    run_cmd(["git", "add", "-A"], verbose=verbose)
    run_cmd(["git", "commit", "-m", message], verbose=verbose)
    run_cmd(["git", "push"], verbose=verbose)
```
Then both functions can call git_sync(message, args.verbose) instead of repeating these three lines.

## Synchronization Workflow

The main sync orchestrator: pull from remote, prepare the project, and push back. Each step validates before proceeding.

In [ ]:
#| export
def sync(args):
    """Sync project: pull, prepare (export/test/clean), commit, and push"""
    print(hr * 60)
    print("PHASE 3: SYNC")
    
    # 1. Git pull
    print("⬇️ 1. Pulling latest changes")
    result = run_cmd(["git", "pull"], check=False, verbose=args.verbose)
    
    if result.returncode != 0:
        status_result = run_cmd(["git", "status", "--porcelain"], capture_output=True, check=False)
        if "UU" in status_result.stdout or "AA" in status_result.stdout:
            print("\n❌ Merge conflict detected!")
            print("   Resolve conflicts manually, then run 'pj sync' again")
            sys.exit(1)
        else:
            print("\n❌ Git pull failed!")
            print("   Fix the issue manually, then try again")
            sys.exit(1)
    
    # 2. nbdev_prepare
    print("🔧 2. Running nbdev_prepare (export, test, clean)")
    prepare_result = run_cmd(["nbdev_prepare"], check=False, capture_output=True)
    
    if prepare_result.returncode != 0:
        print("\n❌ nbdev_prepare failed!")
        print("\nOutput:")
        print(prepare_result.stdout)
        if prepare_result.stderr:
            print("\nErrors:")
            print(prepare_result.stderr)
        print("\n   Fix the errors and try again")
        sys.exit(1)
    
    if args.verbose:
        print(prepare_result.stdout)
    
    # 3. Git commit
    commit_message = args.message or "save"
    print(f"💾 3. Committing changes: '{commit_message}'")
    
    status_result = run_cmd(["git", "status", "--porcelain"], capture_output=True)
    if not status_result.stdout.strip():
        print("   No changes to commit")
    else:
        # TODO: Replace these three lines with git_sync(commit_message, args.verbose)
        run_cmd(["git", "add", "-A"], verbose=args.verbose)
        run_cmd(["git", "commit", "-m", commit_message], verbose=args.verbose)
    
    # 4. Git push
    print("📤 4. Pushing to GitHub")
    run_cmd(["git", "push"], verbose=args.verbose)
    
    print()
    print(hr * 60)
    print("✅ Sync complete!")

### The Sync Gates

**Gate 1: Merge conflict detection**

When `git pull` fails, we inspect `git status --porcelain` for conflict markers. `UU` indicates both sides modified the same file; `AA` means both sides added the same file. These require manual resolution—we abort and tell the user to fix conflicts before retrying.

Other pull failures (network issues, auth problems) also abort but with a generic message since we can't diagnose them automatically.

**Gate 2: nbdev_prepare validation**

The prepare step runs three operations: `nbdev_export` (notebooks → Python modules), `nbdev_test` (run tests), and `nbdev_clean` (strip notebook metadata). If any fail, we capture and display the full output so the user can see exactly which test failed or which export had issues.

We use `capture_output=True` here specifically to show output only on failure. Success is silent (unless `--verbose`); failure is loud.

**Gate 3: Nothing to commit**

After prepare, `git status --porcelain` might show no changes—either nothing was modified, or prepare didn't generate any new files. We detect this and skip the commit step rather than letting git fail with "nothing to commit".

**The commit sequence**

We use `git add -A` to stage all changes: modified files, new files, and deletions. This is essential because `nbdev_export` might create new module files, and `nbdev_clean` might modify notebook metadata. The `-am` shortcut only stages modifications, missing new files entirely.

**TODO: When we extract `git_sync()`, this three-line sequence becomes one function call.** Both `sync()` and `ship()` need this pattern, so factoring it out eliminates duplication.